# Sudoku-10 : Résolution par Réseaux de Neurones

**Navigation** : [<< Sudoku-9-HumanStrategies](Sudoku-9-HumanStrategies.ipynb) | [Index](README.md) | [Sudoku-11-Comparison >>](Sudoku-11-Comparison.ipynb)

**Voir aussi** : [GenAI](../GenAI/README.md) pour les réseaux de neurones

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Encoder une grille de Sudoku sous forme de tenseur adapté à un réseau de neurones
2. Concevoir et entraîner un réseau dense (MLP) pour prédire les chiffres manquants
3. Concevoir et entraîner un réseau convolutif (CNN) qui exploite la structure spatiale
4. Utiliser la prédiction itérative pour améliorer drastiquement la précision
5. Comparer les limites des approches neuronales avec les solveurs algorithmiques

### Prérequis
- Python 3.10+, notions de base en deep learning (couches, loss, backpropagation)
- Familiarité avec NumPy et matplotlib
- Pas besoin de GPU : les modèles sont volontairement légers

### Durée estimée : 55 minutes

### Contexte
Ce notebook s'inspire du projet [jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-CV](https://github.com/jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-CV) (module `Sudoku.NeuralNetwork`) qui explore 4 architectures de réseaux de neurones entraînés sur 17 millions de puzzles. Ici, nous reproduisons les idées clés sur un dataset réduit pour rester exécutable en quelques minutes.

## 1. Introduction : le Sudoku vu comme un probleme de reconnaissance de motifs

Les notebooks precedents de cette serie resolvent le Sudoku par des approches algorithmiques : backtracking, programmation par contraintes, recherche locale. Mais peut-on **apprendre** a resoudre un Sudoku ?

### Pourquoi un reseau de neurones ?

Un reseau de neurones ne "comprend" pas les regles du Sudoku. Il apprend des **correlations statistiques** entre les grilles partielles et leurs solutions a partir de milliers d'exemples. C'est une approche fondamentalement differente :

| Approche | Mecanisme | Garantie de solution valide |
|----------|-----------|-----------------------------|
| **Backtracking** | Exploration exhaustive des possibilites | Oui |
| **CP-SAT / Z3** | Satisfaction de contraintes formelles | Oui |
| **Reseau de neurones** | Apprentissage de motifs a partir de donnees | Non |

### Formulation du probleme

- **Entree** : une grille 9x9 avec des cases vides (valeur 0)
- **Sortie** : pour chaque case, une distribution de probabilite sur les chiffres 1-9
- **Encodage** : one-hot encoding pour transformer les entiers en tenseurs

```
Grille brute :     Tenseur one-hot (9x9x10) :
[5, 0, 0, ...]  -> canal 0 = masque vide
                    canal 5 = 1 en position (0,0)
                    etc.
```

L'idee cle est que le reseau apprend a "deviner" le bon chiffre pour chaque case en s'appuyant sur le contexte spatial (ligne, colonne, bloc 3x3).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from typing import List, Tuple, Optional

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Mode batch pour execution automatisee (Papermill)
BATCH_MODE = os.environ.get('BATCH_MODE', 'false').lower() == 'true'

# Reproductibilite
np.random.seed(42)
torch.manual_seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch {torch.__version__}")
print(f"Device : {device}")
print(f"BATCH_MODE : {BATCH_MODE}")

## 2. Preparation des donnees

Pour entrainer un reseau de neurones, il faut un **dataset** de paires (puzzle, solution). Plutot que de telecharger un fichier externe, nous generons nos donnees de maniere programmatique grace a un solveur par backtracking.

### Strategie de generation

1. **Generer une grille complete valide** : remplir une grille vide par backtracking avec un ordre aleatoire des chiffres
2. **Creer le puzzle** : retirer aleatoirement entre 40 et 55 cases
3. **Stocker la paire** : (puzzle, solution_complete)

### Taille du dataset

Le projet de reference utilise 17 millions de puzzles. Ici, nous travaillons avec **1000 puzzles** pour permettre un entrainement rapide (~1-2 min). C'est suffisant pour observer les comportements fondamentaux des architectures.

### 2.1 Generateur de puzzles

Le generateur utilise un backtracking classique avec une astuce : les chiffres sont testes dans un **ordre aleatoire** a chaque appel, ce qui produit une grille complete differente a chaque execution.

In [ ]:
def is_valid(grid: np.ndarray, row: int, col: int, num: int) -> bool:
    """Verifie si placer num a (row, col) est valide."""
    # Ligne
    if num in grid[row, :]:
        return False
    # Colonne
    if num in grid[:, col]:
        return False
    # Bloc 3x3
    br, bc = 3 * (row // 3), 3 * (col // 3)
    if num in grid[br:br+3, bc:bc+3]:
        return False
    return True


def solve_random(grid: np.ndarray) -> bool:
    """Resout par backtracking avec ordre aleatoire des chiffres."""
    for r in range(9):
        for c in range(9):
            if grid[r, c] == 0:
                digits = list(range(1, 10))
                np.random.shuffle(digits)
                for num in digits:
                    if is_valid(grid, r, c, num):
                        grid[r, c] = num
                        if solve_random(grid):
                            return True
                        grid[r, c] = 0
                return False
    return True


def generate_puzzle(n_remove_min: int = 40, n_remove_max: int = 55) -> Tuple[np.ndarray, np.ndarray]:
    """Genere un puzzle et sa solution.
    
    Returns:
        (puzzle, solution) : grilles 9x9 en entiers 0-9
    """
    # Generer une grille complete
    solution = np.zeros((9, 9), dtype=int)
    solve_random(solution)
    
    # Creer le puzzle en retirant des cases
    puzzle = solution.copy()
    n_remove = np.random.randint(n_remove_min, n_remove_max + 1)
    indices = np.random.choice(81, size=n_remove, replace=False)
    for idx in indices:
        puzzle[idx // 9, idx % 9] = 0
    
    return puzzle, solution


# Test : generer un puzzle
test_puzzle, test_solution = generate_puzzle()
n_empty = np.sum(test_puzzle == 0)
print(f"Puzzle genere : {n_empty} cases vides")
print("\nPuzzle :")
print(test_puzzle)
print("\nSolution :")
print(test_solution)

### 2.2 Construction du dataset

Nous generons 1000 puzzles et les separons en 800 pour l'entrainement et 200 pour le test.

**Encodage one-hot** :
- **Puzzle** : tenseur `(9, 9, 10)` -- le canal 0 represente "case vide", les canaux 1-9 representent les chiffres
- **Solution** : tenseur `(9, 9)` avec les chiffres 1-9 decales en indices 0-8 pour la cross-entropy

Cet encodage permet au reseau de distinguer explicitement une case vide (a predire) d'une case contenant le chiffre 0 (qui n'existe pas en Sudoku).

In [ ]:
def encode_puzzle(puzzle: np.ndarray) -> np.ndarray:
    """Encode un puzzle en one-hot (9, 9, 10).
    Canal 0 = case vide, canaux 1-9 = chiffres."""
    encoded = np.zeros((9, 9, 10), dtype=np.float32)
    for r in range(9):
        for c in range(9):
            encoded[r, c, puzzle[r, c]] = 1.0
    return encoded


def encode_solution(solution: np.ndarray) -> np.ndarray:
    """Encode la solution en indices 0-8 (pour cross-entropy)."""
    return (solution - 1).astype(np.int64)  # chiffres 1-9 -> indices 0-8


# Generer le dataset
N_SAMPLES = 1000
print(f"Generation de {N_SAMPLES} puzzles...")

start = time.time()
puzzles_raw = []
solutions_raw = []

for i in range(N_SAMPLES):
    p, s = generate_puzzle()
    puzzles_raw.append(p)
    solutions_raw.append(s)
    if (i + 1) % 200 == 0:
        print(f"  {i + 1}/{N_SAMPLES} generes...")

elapsed = time.time() - start
print(f"Generation terminee en {elapsed:.1f}s")

# Encoder
X_all = np.array([encode_puzzle(p) for p in puzzles_raw])    # (N, 9, 9, 10)
y_all = np.array([encode_solution(s) for s in solutions_raw]) # (N, 9, 9)

# Split train/test
N_TRAIN = 800
X_train, X_test = X_all[:N_TRAIN], X_all[N_TRAIN:]
y_train, y_test = y_all[:N_TRAIN], y_all[N_TRAIN:]
puzzles_test = puzzles_raw[N_TRAIN:]
solutions_test = solutions_raw[N_TRAIN:]

print(f"\nDataset :")
print(f"  Train : {X_train.shape[0]} puzzles")
print(f"  Test  : {X_test.shape[0]} puzzles")
print(f"  X shape : {X_train.shape}")
print(f"  y shape : {y_train.shape}")

### Interpretation : encodage des donnees

| Element | Shape | Description |
|---------|-------|-------------|
| Puzzle brut | `(9, 9)` | Entiers 0-9, 0 = case vide |
| Puzzle encode | `(9, 9, 10)` | One-hot, canal 0 = vide |
| Solution cible | `(9, 9)` | Indices 0-8 (chiffre - 1) |

Le one-hot encoding est essentiel : sans lui, le reseau traiterait les chiffres comme des valeurs ordonnees (5 > 3) alors qu'en Sudoku ils sont simplement des **etiquettes** sans relation d'ordre.

In [ ]:
class SudokuDataset(Dataset):
    """Dataset PyTorch pour les puzzles de Sudoku."""
    
    def __init__(self, X: np.ndarray, y: np.ndarray):
        # Reorganiser en (N, channels, H, W) pour PyTorch
        self.X = torch.FloatTensor(X).permute(0, 3, 1, 2)  # (N, 10, 9, 9)
        self.y = torch.LongTensor(y)                        # (N, 9, 9)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Creer les DataLoaders
BATCH_SIZE = 64

train_dataset = SudokuDataset(X_train, y_train)
test_dataset = SudokuDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Verifier les dimensions
X_batch, y_batch = next(iter(train_loader))
print(f"Batch X : {X_batch.shape}")  # (64, 10, 9, 9)
print(f"Batch y : {y_batch.shape}")  # (64, 9, 9)

## 3. Architecture 1 : Reseau Dense (MLP)

La premiere approche est la plus naive : aplatir la grille en un vecteur et utiliser des couches denses (fully connected). Le reseau traite la grille comme un vecteur de 810 valeurs (9 x 9 x 10) et produit 729 sorties (9 x 9 x 9 = une distribution sur 9 chiffres pour chaque case).

### Architecture

```
Input(810) -> Dense(512) -> ReLU -> Dense(512) -> ReLU -> Dense(729) -> Reshape(9, 9, 9)
```

### Limitations attendues

Le MLP n'a **aucune notion de structure spatiale**. Il ne sait pas que la case (0,0) et la case (0,8) sont dans la meme ligne, ni que les cases forment des blocs 3x3. Tout est un vecteur plat. Malgre cela, il peut apprendre certains motifs statistiques.

In [ ]:
class DenseModel(nn.Module):
    """Reseau dense (MLP) pour la resolution de Sudoku."""
    
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(9 * 9 * 10, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.Linear(512, 9 * 9 * 9)
        )
    
    def forward(self, x):
        # x: (batch, 10, 9, 9)
        x = self.flatten(x)              # (batch, 810)
        x = self.network(x)              # (batch, 729)
        x = x.view(-1, 9, 9, 9)          # (batch, 9, 9, 9)
        return x


dense_model = DenseModel().to(device)
n_params = sum(p.numel() for p in dense_model.parameters())
print(f"DenseModel : {n_params:,} parametres")
print(dense_model)

### Fonction d'entrainement et d'evaluation

Nous utilisons la **cross-entropy** comme fonction de perte : pour chaque case, le reseau produit 9 logits et la perte mesure l'ecart avec le bon chiffre. Deux metriques sont calculees :

- **Precision par case** : pourcentage de cases individuelles correctement predites
- **Precision par grille** : pourcentage de grilles entierement correctes (les 81 cases justes)

In [ ]:
def train_model(model, train_loader, test_loader, n_epochs=20, lr=1e-3):
    """Entraine un modele et retourne l'historique."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {
        'train_loss': [], 'test_loss': [],
        'cell_acc': [], 'grid_acc': []
    }
    
    for epoch in range(n_epochs):
        # --- Entrainement ---
        model.train()
        train_loss = 0.0
        n_batches = 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            output = model(X_batch)       # (batch, 9, 9, 9)
            # Reshape pour cross-entropy : (batch*81, 9) vs (batch*81)
            loss = criterion(
                output.view(-1, 9),
                y_batch.view(-1)
            )
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            n_batches += 1
        
        train_loss /= n_batches
        
        # --- Evaluation ---
        model.eval()
        test_loss = 0.0
        correct_cells = 0
        total_cells = 0
        correct_grids = 0
        total_grids = 0
        n_batches_test = 0
        
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                output = model(X_batch)
                loss = criterion(output.view(-1, 9), y_batch.view(-1))
                test_loss += loss.item()
                n_batches_test += 1
                
                # Predictions
                preds = output.argmax(dim=-1)  # (batch, 9, 9)
                correct_cells += (preds == y_batch).sum().item()
                total_cells += y_batch.numel()
                
                # Grilles entierement correctes
                grid_correct = (preds == y_batch).all(dim=-1).all(dim=-1)
                correct_grids += grid_correct.sum().item()
                total_grids += y_batch.shape[0]
        
        test_loss /= n_batches_test
        cell_acc = correct_cells / total_cells
        grid_acc = correct_grids / total_grids
        
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['cell_acc'].append(cell_acc)
        history['grid_acc'].append(grid_acc)
        
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:2d}/{n_epochs} | "
                  f"Loss train={train_loss:.4f} test={test_loss:.4f} | "
                  f"Cell acc={cell_acc:.3f} | Grid acc={grid_acc:.3f}")
    
    return history


def plot_history(history, title=""):
    """Affiche les courbes d'entrainement."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].plot(history['train_loss'], label='Train')
    axes[0].plot(history['test_loss'], label='Test')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Fonction de perte')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history['cell_acc'])
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Precision par case')
    axes[1].set_ylim(0, 1)
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(history['grid_acc'])
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Precision')
    axes[2].set_title('Precision par grille complete')
    axes[2].set_ylim(0, 1)
    axes[2].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

### Entrainement du MLP

Nous entrainons le reseau dense pendant 20 epochs avec l'optimiseur Adam. Observez l'ecart entre la precision par case (relativement elevee) et la precision par grille (beaucoup plus basse) : il suffit d'une seule case fausse pour qu'une grille entiere soit comptee comme incorrecte.

In [ ]:
N_EPOCHS = 20

print("=== Entrainement du reseau dense (MLP) ===")
dense_model = DenseModel().to(device)

start = time.time()
dense_history = train_model(dense_model, train_loader, test_loader, n_epochs=N_EPOCHS)
dense_time = time.time() - start

print(f"\nTemps d'entrainement : {dense_time:.1f}s")
plot_history(dense_history, "Reseau Dense (MLP)")

### Interpretation : performances du MLP

**Observations attendues** :

| Metrique | Valeur typique | Explication |
|----------|---------------|-------------|
| Precision par case | ~70-85% | Le reseau apprend les motifs frequents |
| Precision par grille | ~0-5% | Tres peu de grilles entierement correctes |

**Points cles** :
1. Le MLP peut atteindre une bonne precision par case, mais cela ne suffit pas : chaque erreur se propage et invalide la grille
2. Sans notion de structure spatiale, le reseau ne peut pas exploiter les contraintes de lignes, colonnes et blocs
3. La precision par grille reste tres faible car 81 predictions doivent etre simultanement correctes

> **Note technique** : sur 1000 puzzles d'entrainement, le MLP apprend essentiellement les cases les plus faciles (celles contraintes par beaucoup d'indices). Les cases ambigues restent difficiles.

## 4. Architecture 2 : Reseau Convolutif (CNN)

Les reseaux convolutifs sont connus pour leur capacite a capturer les **motifs spatiaux locaux**. En traitant la grille de Sudoku comme une "image" a 10 canaux, les filtres de convolution peuvent apprendre les interactions entre cases voisines.

### Pourquoi la convolution est pertinente

Les contraintes du Sudoku sont **locales** :
- Les cases d'un meme bloc 3x3 interagissent (filtre 3x3 capture exactement un bloc)
- Les cases d'une ligne/colonne interagissent (filtres empiles capturent des contextes plus larges)

### Architecture

```
Input(10, 9, 9)
  -> Conv2D(64, 3x3, pad=same) -> BN -> ReLU
  -> Conv2D(128, 3x3, pad=same) -> BN -> ReLU
  -> Conv2D(128, 3x3, pad=same) -> BN -> ReLU
  -> Conv2D(256, 3x3, pad=same) -> BN -> ReLU
  -> Conv2D(256, 3x3, pad=same) -> BN -> ReLU
  -> Conv2D(9, 1x1) -> Permute
Output(9, 9, 9)
```

Le `padding='same'` preserve la resolution spatiale (9x9) tout au long du reseau. La derniere couche `1x1` produit 9 canaux (un par chiffre possible) pour chaque case.

In [ ]:
class CNNModel(nn.Module):
    """Reseau convolutif pour la resolution de Sudoku."""
    
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            # Couche 1 : capturer les motifs locaux
            nn.Conv2d(10, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            # Couche 2 : combiner les motifs
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            # Couche 3 : contexte plus large
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            # Couche 4 : representation riche
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            # Couche 5 : affiner les predictions
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            # Couche de sortie : 1x1 conv pour produire 9 classes
            nn.Conv2d(256, 9, kernel_size=1)
        )
    
    def forward(self, x):
        # x: (batch, 10, 9, 9)
        x = self.conv_layers(x)           # (batch, 9, 9, 9)
        x = x.permute(0, 2, 3, 1)         # (batch, 9, 9, 9) - dernier dim = classes
        return x


cnn_model = CNNModel().to(device)
n_params = sum(p.numel() for p in cnn_model.parameters())
print(f"CNNModel : {n_params:,} parametres")
print(cnn_model)

### Entrainement du CNN

Le CNN est entraine dans les memes conditions que le MLP (20 epochs, meme dataset). L'objectif est de comparer equitablement les deux architectures.

In [ ]:
print("=== Entrainement du reseau convolutif (CNN) ===")
cnn_model = CNNModel().to(device)

start = time.time()
cnn_history = train_model(cnn_model, train_loader, test_loader, n_epochs=N_EPOCHS)
cnn_time = time.time() - start

print(f"\nTemps d'entrainement : {cnn_time:.1f}s")
plot_history(cnn_history, "Reseau Convolutif (CNN)")

### Interpretation : CNN vs MLP

**Comparaison directe** :

| Metrique | MLP | CNN | Avantage CNN |
|----------|-----|-----|-------|
| Precision par case | ~70-85% | ~80-92% | Structure spatiale exploitee |
| Precision par grille | ~0-5% | ~5-15% | Meilleur mais toujours faible |
| Nombre de parametres | ~700K | ~500K | CNN plus compact |

**Points cles** :
1. Le CNN surpasse le MLP grace aux filtres 3x3 qui capturent naturellement les blocs du Sudoku
2. Avec 5 couches de convolution et `padding=same`, le champ receptif couvre une grande partie de la grille
3. Malgre l'amelioration, la precision par grille reste modeste : le reseau fait encore des erreurs occasionnelles

> **Note technique** : avec le dataset de reference de 17M puzzles, ces architectures atteignent des precisions par case de 95%+. Notre dataset reduit (1000 puzzles) limite la generalisation.

## 5. Prediction iterative : resoudre case par case

L'idee cle pour ameliorer drastiquement la precision est d'imiter la strategie humaine : **predire la case la plus certaine, la remplir, puis re-predire**.

### Principe

Au lieu de predire les 81 cases d'un coup, on procede iterativement :

1. Le reseau predit les probabilites pour toutes les cases vides
2. On selectionne la prediction la plus confiante (probabilite maximale la plus elevee)
3. On remplit cette case dans le puzzle
4. On re-encode le puzzle mis a jour et on recommence
5. On repete jusqu'a ce que toutes les cases soient remplies

### Pourquoi cela fonctionne

Chaque case remplie fournit un **indice supplementaire** au reseau pour les cases restantes. C'est un mecanisme d'auto-regression : les predictions les plus sures servent de base pour les predictions suivantes.

```
Iteration 1 : 45 cases vides -> predire la plus certaine (confiance 99%)
Iteration 2 : 44 cases vides -> plus d'indices, predictions plus sures
...
Iteration 45 : 1 case vide -> presque triviale
```

**Risque** : si une prediction intermediaire est fausse, les erreurs se propagent. Le seuil de confiance peut attenuer ce risque.

In [ ]:
def iterative_predict(model, puzzle: np.ndarray, verbose: bool = False) -> np.ndarray:
    """Prediction iterative : remplir une case a la fois.
    
    Args:
        model: Modele entraine
        puzzle: Grille 9x9 (0 = vide)
        verbose: Afficher les etapes intermediaires
    
    Returns:
        Grille 9x9 completee
    """
    model.eval()
    current = puzzle.copy()
    steps = []
    
    while np.any(current == 0):
        # Encoder le puzzle courant
        encoded = encode_puzzle(current)
        x = torch.FloatTensor(encoded).permute(2, 0, 1).unsqueeze(0).to(device)  # (1, 10, 9, 9)
        
        with torch.no_grad():
            output = model(x)  # (1, 9, 9, 9)
            probs = torch.softmax(output[0], dim=-1)  # (9, 9, 9)
        
        probs = probs.cpu().numpy()
        
        # Trouver la case vide avec la prediction la plus confiante
        best_conf = -1
        best_pos = None
        best_digit = None
        
        for r in range(9):
            for c in range(9):
                if current[r, c] == 0:
                    max_prob = probs[r, c].max()
                    if max_prob > best_conf:
                        best_conf = max_prob
                        best_pos = (r, c)
                        best_digit = probs[r, c].argmax() + 1  # indice 0-8 -> chiffre 1-9
        
        if best_pos is None:
            break
        
        current[best_pos[0], best_pos[1]] = best_digit
        steps.append((best_pos, best_digit, best_conf))
        
        if verbose and len(steps) <= 5:
            print(f"  Etape {len(steps):2d} : case ({best_pos[0]},{best_pos[1]}) "
                  f"= {best_digit} (confiance {best_conf:.3f})")
    
    if verbose:
        if len(steps) > 5:
            print(f"  ... ({len(steps) - 5} etapes supplementaires)")
        print(f"  Total : {len(steps)} cases remplies")
    
    return current, steps


# Test sur quelques puzzles
print("=== Prediction iterative avec le CNN ===")
print()

for i in range(3):
    puzzle = puzzles_test[i]
    solution = solutions_test[i]
    n_empty = np.sum(puzzle == 0)
    
    print(f"Puzzle {i+1} ({n_empty} cases vides) :")
    
    # Prediction directe (en une passe)
    encoded = encode_puzzle(puzzle)
    x = torch.FloatTensor(encoded).permute(2, 0, 1).unsqueeze(0).to(device)
    with torch.no_grad():
        direct_output = cnn_model(x)
        direct_preds = direct_output[0].argmax(dim=-1).cpu().numpy() + 1
    direct_correct = np.sum(direct_preds == solution)
    
    # Prediction iterative
    iterative_result, steps = iterative_predict(cnn_model, puzzle, verbose=True)
    iter_correct = np.sum(iterative_result == solution)
    
    print(f"  Directe   : {direct_correct}/81 cases correctes ({direct_correct == 81})")
    print(f"  Iterative : {iter_correct}/81 cases correctes ({iter_correct == 81})")
    print()

### Interpretation : impact de la prediction iterative

La prediction iterative ameliore considerablement les resultats car :

| Aspect | Prediction directe | Prediction iterative |
|--------|-------------------|---------------------|
| Contexte utilise | Uniquement les indices initiaux | Indices + cases deja remplies |
| Risque d'erreur | Independant par case | Risque de propagation d'erreurs |
| Precision par grille | Faible | Nettement superieure |

**Points cles** :
1. La confiance du reseau est un bon indicateur : les premieres cases remplies sont souvent correctes
2. L'amelioration est d'autant plus forte que le modele est precis par case
3. C'est exactement le mecanisme utilise par les humains : resoudre d'abord les cases evidentes

> **Note technique** : cette approche est similaire aux modeles auto-regressifs en NLP (GPT) qui generent un token a la fois en conditionnant sur les tokens precedents.

## 6. Evaluation comparative et analyse

Evaluons systematiquement les deux architectures (MLP et CNN) avec les deux strategies de prediction (directe et iterative) sur le jeu de test complet.

In [ ]:
def evaluate_model(model, puzzles, solutions, use_iterative=False):
    """Evalue un modele sur un ensemble de puzzles.
    
    Returns:
        dict avec cell_acc, grid_acc, et details par puzzle
    """
    model.eval()
    total_correct_cells = 0
    total_cells = 0
    correct_grids = 0
    n_puzzles = len(puzzles)
    
    for puzzle, solution in zip(puzzles, solutions):
        if use_iterative:
            prediction, _ = iterative_predict(model, puzzle)
        else:
            encoded = encode_puzzle(puzzle)
            x = torch.FloatTensor(encoded).permute(2, 0, 1).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(x)
                prediction = output[0].argmax(dim=-1).cpu().numpy() + 1
        
        n_correct = np.sum(prediction == solution)
        total_correct_cells += n_correct
        total_cells += 81
        if n_correct == 81:
            correct_grids += 1
    
    return {
        'cell_acc': total_correct_cells / total_cells,
        'grid_acc': correct_grids / n_puzzles,
        'correct_grids': correct_grids,
        'total': n_puzzles
    }


# Evaluation complete
print("=== Evaluation sur le jeu de test (200 puzzles) ===")
print()

results = {}

print("MLP - prediction directe...")
results['MLP direct'] = evaluate_model(dense_model, puzzles_test, solutions_test, use_iterative=False)

print("MLP - prediction iterative...")
results['MLP iteratif'] = evaluate_model(dense_model, puzzles_test, solutions_test, use_iterative=True)

print("CNN - prediction directe...")
results['CNN direct'] = evaluate_model(cnn_model, puzzles_test, solutions_test, use_iterative=False)

print("CNN - prediction iterative...")
results['CNN iteratif'] = evaluate_model(cnn_model, puzzles_test, solutions_test, use_iterative=True)

# Tableau recapitulatif
print()
print(f"{'Modele':<20} {'Precision/case':>15} {'Precision/grille':>18} {'Grilles OK':>12}")
print("-" * 70)
for name, r in results.items():
    print(f"{name:<20} {r['cell_acc']:>14.1%} {r['grid_acc']:>17.1%} {r['correct_grids']:>8}/{r['total']}")

### Interpretation : tableau comparatif

| Configuration | Precision/case | Precision/grille | Observation |
|---------------|---------------|-----------------|-------------|
| MLP direct | ~75% | ~0-2% | Baseline naive |
| MLP iteratif | ~85% | ~5-15% | Amelioration modeste |
| CNN direct | ~85% | ~5-10% | Structure spatiale aide |
| CNN iteratif | ~90-95% | ~20-50% | Meilleure combinaison |

**Points cles** :
1. La prediction iterative apporte un gain significatif pour les deux architectures
2. Le CNN beneficie davantage de l'iteratif car ses predictions individuelles sont plus fiables
3. Meme le meilleur resultat est loin de 100% -- un solveur algorithmique (backtracking, CP-SAT) resout 100% des puzzles valides

In [ ]:
# Visualisation : grille predite vs solution
def plot_sudoku_comparison(puzzle, prediction, solution, title=""):
    """Affiche puzzle, prediction et solution cote a cote."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    titles = ["Puzzle", "Prediction NN", "Solution"]
    grids = [puzzle, prediction, solution]
    
    for ax, grid, t in zip(axes, grids, titles):
        ax.set_xlim(0, 9)
        ax.set_ylim(0, 9)
        ax.set_aspect('equal')
        ax.set_title(t, fontsize=12)
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Lignes de grille
        for i in range(10):
            lw = 2 if i % 3 == 0 else 0.5
            ax.axhline(i, color='black', linewidth=lw)
            ax.axvline(i, color='black', linewidth=lw)
        
        # Chiffres
        for r in range(9):
            for c in range(9):
                val = grid[r, c]
                if val != 0:
                    # Couleur : noir=indice, bleu=predit correct, rouge=predit faux
                    if puzzle[r, c] != 0:
                        color = 'black'
                    elif grid is prediction:
                        color = 'blue' if val == solution[r, c] else 'red'
                    else:
                        color = 'blue'
                    
                    ax.text(c + 0.5, 8.5 - r, str(val),
                           ha='center', va='center', fontsize=11, color=color,
                           fontweight='bold' if puzzle[r, c] != 0 else 'normal')
    
    if title:
        plt.suptitle(title, fontsize=13, y=1.02)
    plt.tight_layout()
    plt.show()


# Afficher 2 exemples
for i in range(2):
    puzzle = puzzles_test[i]
    solution = solutions_test[i]
    
    prediction, _ = iterative_predict(cnn_model, puzzle)
    n_correct = np.sum(prediction == solution)
    n_empty = np.sum(puzzle == 0)
    
    plot_sudoku_comparison(
        puzzle, prediction, solution,
        f"Puzzle {i+1} : {n_correct}/81 cases correctes ({n_empty} vides)"
    )

In [ ]:
# Analyse des erreurs : quelles cases sont les plus difficiles ?
error_map = np.zeros((9, 9), dtype=int)

for puzzle, solution in zip(puzzles_test, solutions_test):
    prediction, _ = iterative_predict(cnn_model, puzzle)
    errors = (prediction != solution).astype(int)
    # Ne compter les erreurs que sur les cases vides du puzzle
    mask_empty = (puzzle == 0).astype(int)
    error_map += errors * mask_empty

# Heatmap des erreurs
fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(error_map, cmap='YlOrRd', aspect='equal')
ax.set_title("Carte des erreurs de prediction (CNN iteratif)", fontsize=12)
ax.set_xlabel("Colonne")
ax.set_ylabel("Ligne")

# Ajouter les valeurs dans les cases
for r in range(9):
    for c in range(9):
        ax.text(c, r, str(error_map[r, c]),
               ha='center', va='center', fontsize=10,
               color='white' if error_map[r, c] > error_map.max() * 0.5 else 'black')

# Lignes de bloc
for i in [3, 6]:
    ax.axhline(i - 0.5, color='black', linewidth=2)
    ax.axvline(i - 0.5, color='black', linewidth=2)

plt.colorbar(im, label="Nombre d'erreurs")
plt.tight_layout()
plt.show()

print(f"\nTotal erreurs : {error_map.sum()}")
print(f"Position la plus difficile : ligne {error_map.argmax() // 9}, colonne {error_map.argmax() % 9} ({error_map.max()} erreurs)")

### Interpretation : carte des erreurs

La heatmap revele les positions ou le reseau commet le plus d'erreurs :

**Points cles** :
1. Les erreurs ne sont pas uniformement reparties : certaines positions sont systematiquement plus difficiles
2. Les cases au centre de la grille sont souvent plus difficiles car elles sont contraintes par plus de voisins
3. Le reseau n'a pas de mecanisme pour **verifier la validite** de ses predictions (pas de contrainte AllDifferent)

### Limitations fondamentales

| Limitation | Consequence | Solution possible |
|-----------|-------------|------------------|
| Pas de contraintes explicites | Grilles invalides possibles | Hybrid NN + verificateur |
| Dataset limite (1000) | Generalisation insuffisante | Plus de donnees (17M) |
| Propagation d'erreurs | Erreur precoce = grille fausse | Seuil de confiance + backtrack |

> **Vers une approche hybride** : combiner la prediction neuronale avec un verificateur de contraintes permet de garantir la validite. Le reseau propose, le solveur dispose.

## 7. Exercices

### Exercice 1 : Experimenter avec l'architecture CNN

Modifiez l'architecture du CNN pour observer l'impact :
- Ajouter ou retirer des couches de convolution
- Modifier le nombre de filtres (32, 64, 128, 256)
- Tester des kernel sizes differents (1x1, 3x3, 5x5)

**Question** : quel est le nombre minimal de couches pour atteindre un champ receptif couvrant toute la grille 9x9 avec des filtres 3x3 ?

In [ ]:
# Exercice 1 : Experimentez ici
# Indice : le champ receptif d'un filtre 3x3 apres n couches est (2n+1) x (2n+1)
# Pour couvrir 9x9, il faut 2n+1 >= 9, soit n >= 4 couches

# Exemple : CNN plus profond
# class DeepCNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         layers = []
#         in_channels = 10
#         for out_channels in [64, 64, 128, 128, 256, 256, 256]:
#             layers.extend([
#                 nn.Conv2d(in_channels, out_channels, 3, padding=1),
#                 nn.BatchNorm2d(out_channels),
#                 nn.ReLU()
#             ])
#             in_channels = out_channels
#         layers.append(nn.Conv2d(in_channels, 9, 1))
#         self.net = nn.Sequential(*layers)
#     
#     def forward(self, x):
#         return self.net(x).permute(0, 2, 3, 1)

### Exercice 2 : Dropout et regularisation

Ajoutez du dropout et de la batch normalization au MLP et au CNN. Comparez les courbes de train/test loss pour detecter le sur-apprentissage.

**Questions** :
- Le dropout ameliore-t-il la precision sur le jeu de test ?
- A partir de combien d'epochs observe-t-on du sur-apprentissage ?

In [ ]:
# Exercice 2 : Experimentez ici
# Indice : ajoutez nn.Dropout2d(0.2) apres les couches ReLU du CNN
# Comparez les courbes train_loss et test_loss

# Exemple :
# class CNNWithDropout(nn.Module):
#     def __init__(self, dropout_rate=0.2):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Conv2d(10, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
#             nn.Dropout2d(dropout_rate),
#             nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
#             nn.Dropout2d(dropout_rate),
#             nn.Conv2d(128, 9, 1)
#         )
#     def forward(self, x):
#         return self.net(x).permute(0, 2, 3, 1)

### Exercice 3 : Approche hybride NN + backtracking

Implementez une approche hybride :
1. Utilisez le CNN pour predire les chiffres les plus probables
2. Remplissez les cases ou la confiance depasse un seuil (ex: 0.95)
3. Pour les cases restantes, utilisez le backtracking classique

**Question** : cette approche garantit-elle une solution valide ? Quel est l'avantage par rapport au backtracking seul ?

In [ ]:
# Exercice 3 : Approche hybride
# Indice : le NN reduit le nombre de cases a explorer par backtracking

def hybrid_solve(model, puzzle: np.ndarray, confidence_threshold: float = 0.95) -> np.ndarray:
    """Resout un Sudoku par approche hybride NN + backtracking."""
    current = puzzle.copy()
    
    # Phase 1 : remplir les cases avec haute confiance
    model.eval()
    nn_filled = 0
    
    while True:
        encoded = encode_puzzle(current)
        x = torch.FloatTensor(encoded).permute(2, 0, 1).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(x)
            probs = torch.softmax(output[0], dim=-1).cpu().numpy()
        
        # Trouver la case vide la plus confiante
        best_conf = -1
        best_pos = None
        best_digit = None
        
        for r in range(9):
            for c in range(9):
                if current[r, c] == 0:
                    max_prob = probs[r, c].max()
                    if max_prob > best_conf:
                        best_conf = max_prob
                        best_pos = (r, c)
                        best_digit = probs[r, c].argmax() + 1
        
        if best_pos is None or best_conf < confidence_threshold:
            break
        
        current[best_pos[0], best_pos[1]] = best_digit
        nn_filled += 1
    
    # Phase 2 : backtracking pour les cases restantes
    remaining = np.sum(current == 0)
    
    def backtrack(grid):
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    for num in range(1, 10):
                        if is_valid(grid, r, c, num):
                            grid[r, c] = num
                            if backtrack(grid):
                                return True
                            grid[r, c] = 0
                    return False
        return True
    
    backtrack(current)
    print(f"  NN a rempli {nn_filled} cases (seuil={confidence_threshold}), "
          f"backtracking pour les {remaining} restantes")
    
    return current


# Test
print("=== Approche hybride CNN + Backtracking ===")
for i in range(3):
    puzzle = puzzles_test[i]
    solution = solutions_test[i]
    print(f"\nPuzzle {i+1} :")
    result = hybrid_solve(cnn_model, puzzle, confidence_threshold=0.90)
    n_correct = np.sum(result == solution)
    print(f"  Resultat : {n_correct}/81 cases correctes")

## 8. Conclusion

### Recapitulatif

| Aspect | MLP | CNN | CNN iteratif | Hybride NN+BT |
|--------|-----|-----|-------------|----------------|
| Structure spatiale | Non | Oui | Oui | Oui |
| Precision/case | ~75% | ~85% | ~90-95% | 100%* |
| Precision/grille | ~0-2% | ~5-10% | ~20-50% | 100%* |
| Garantie solution | Non | Non | Non | Oui |
| Temps | ~ms | ~ms | ~100ms | Variable |

\* L'approche hybride garantit une solution valide grace au backtracking de secours.

### Ce que nous avons appris

1. **Les reseaux de neurones peuvent apprendre les motifs du Sudoku** mais ne garantissent pas des solutions valides
2. **La structure spatiale compte** : le CNN surpasse le MLP grace aux convolutions
3. **La prediction iterative est cruciale** : elle transforme un classificateur moyen en un solveur potable
4. **L'approche hybride est le meilleur compromis** : la vitesse du NN + la fiabilite du backtracking

### Comparaison avec les autres approches de la serie

| Solveur | Type | Fiabilite | Rapidite |
|---------|------|-----------|----------|
| Backtracking | Algorithmique | 100% | Rapide |
| OR-Tools/Z3 | Contraintes | 100% | Tres rapide |
| Dancing Links | Couverture exacte | 100% | Optimal |
| Norvig | Propagation + BT | 100% | Rapide |
| Genetique | Metaheuristique | ~50% | Lent |
| **Reseau de neurones** | **Apprentissage** | **Variable** | **Rapide** |

Les reseaux de neurones ne sont pas la meilleure approche pour resoudre le Sudoku (les solveurs par contraintes sont plus fiables et souvent plus rapides). Leur interet est **pedagogique** : ils illustrent comment le deep learning peut etre applique a des problemes combinatoires, et quelles sont ses limites fondamentales.

### Pour aller plus loin

- [Projet de reference : 4 architectures sur 17M puzzles](https://github.com/jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-CV)
- [Park (2018) : Relational recurrent neural network for Sudoku](https://arxiv.org/abs/1802.04310)
- [Graph Neural Networks for combinatorial optimization](https://arxiv.org/abs/2012.01806)
- Approches par reinforcement learning : apprendre a resoudre par essai-erreur

---

**Navigation** : [<< Sudoku-9-HumanStrategies](Sudoku-9-HumanStrategies.ipynb) | [Index](README.md) | [Sudoku-11-Comparison >>](Sudoku-11-Comparison.ipynb)

**Voir aussi** : 
- [GenAI](../GenAI/README.md) - Série sur l'IA générative et les réseaux de neurones
- [Sudoku-11-Comparison](Sudoku-11-Comparison.ipynb) - Benchmark comparatif de tous les solveurs

### Notebooks associés
- [Sudoku-1-Backtracking](Sudoku-1-Backtracking.ipynb) : solveur algorithmique de référence
- [Sudoku-Python-Backtracking](Sudoku-Python-Backtracking.ipynb) : backtracking en Python (meme SudokuGrid)
- [Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) : propagation de contraintes
- [Sudoku-Python-Genetic](Sudoku-Python-Genetic.ipynb) : autre approche non-déterministe